In [1]:
import os
import numpy as np
import pandas as pd
import gzip
import requests
import time
from multiprocessing import Pool
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
import json
from hashlib import md5
import random
import datetime
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import torch

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df_stru = pd.read_csv("MIMIC3_CC_V1.csv")
df_stru

,subject_id,hadm_id,admittime,dischtime,edregtime,edouttime,diagnosis,hospital_expire_flag,gender,age,hospital_days,Respiratory rate,Heart rate,Systolic blood pressure,Diastolic blood pressure,Body temperature,Saturation of peripheral oxygen,Glasgow coma scale - eye,Glasgow coma scale - verbal,Glasgow coma scale - motor,Alanine aminotransferase,Aspartate aminotransferase,Albumin,Blood urea nitrogen,Bilirubin,C-reactive protein,Chloride,Creatinine,Glucose,Hemoglobin,Prothrombin time (INR),Platelets,Potassium,Sodium,White blood cell,Activated partial thromboplastin time
0,9,150750,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-09 11:13:00,2149-11-09 13:18:00,HEMORRHAGIC CVA,1,M,41.8,4.9,17.542857,83.771429,133.588235,71.000000,36.096000,98.285714,1.000000,1.000000,1.083333,NaN,NaN,NaN,30.500000,NaN,NaN,102.500000,1.850000,149.000000,12.050000,1.10,318.000000,3.600000,133.400000,11.650000,23.300000
1,11,194540,2178-04-16 06:18:00,2178-05-11 19:00:00,2178-04-15 20:46:00,2178-04-16 06:53:00,BRAIN MASS,0,F,50.1,25.5,18.176471,84.918919,106.666667,54.194444,36.768750,96.655172,4.000000,4.941176,5.882353,NaN,NaN,NaN,16.500000,NaN,NaN,106.000000,0.700000,128.333333,12.550000,NaN,250.500000,3.850000,141.500000,11.050000,NaN
2,13,143045,2167-01-08 18:43:00,2167-01-15 15:15:00,NaN,NaN,CORONARY ARTERY DISEASE,0,F,39.9,6.9,17.646552,98.017241,107.250000,48.500000,37.122500,98.592920,3.782609,4.304348,5.608696,16.0,16.0,3.90,14.833333,NaN,NaN,107.200000,0.583333,158.833333,10.820000,1.40,135.500000,4.175000,138.375000,17.200000,33.875000
3,19,109235,2108-08-05 16:25:00,2108-08-11 11:29:00,2108-08-05 08:01:00,2108-08-05 20:12:00,C-2 FRACTURE,0,M,91.0,5.8,20.583333,98.416667,146.708333,62.000000,36.872857,98.208333,4.000000,4.200000,6.000000,NaN,NaN,NaN,25.000000,NaN,NaN,105.000000,1.200000,139.000000,12.800000,1.30,216.000000,4.200000,138.000000,10.100000,30.400000
4,20,157681,2183-04-28 09:45:00,2183-05-03 14:45:00,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,F,75.9,5.2,15.512821,79.121951,118.333333,42.166667,36.000000,98.435897,3.000000,3.000000,4.333333,NaN,NaN,NaN,18.000000,NaN,NaN,108.000000,0.800000,153.750000,9.500000,1.45,121.500000,4.088889,137.500000,17.500000,32.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14399,32786,165351,2152-11-15 20:26:00,2152-12-08 11:00:00,2152-11-15 18:39:00,2152-11-15 22:21:00,SUBARACHNOID HEMORRHAGE,0,M,53.7,22.6,22.461314,68.940666,152.684211,55.041509,37.954350,97.714286,3.112676,1.706714,4.616197,67.0,47.5,3.44,34.793103,NaN,NaN,105.275862,0.927586,161.782609,10.844444,1.10,448.740741,4.330303,139.896552,16.648148,26.382609
14400,32786,163011,2152-12-09 19:17:00,2152-12-15 14:30:00,2152-12-09 15:28:00,2152-12-09 22:54:00,ALTERED MENTAL STATUS,0,M,53.8,5.8,23.518519,61.888889,131.423077,66.192308,37.230000,96.923077,2.000000,1.000000,1.857143,475.0,269.5,3.70,82.000000,NaN,NaN,107.000000,1.350000,159.000000,10.800000,1.20,457.000000,4.650000,147.000000,14.150000,25.700000
14401,32787,157308,2102-10-02 07:15:00,2102-10-09 16:42:00,NaN,NaN,AORTIC STENOSIS\AORTIC AND MITRAL VALVE REPLAC...,0,M,44.7,7.4,16.694737,81.526316,115.500000,71.904762,37.500000,97.543478,3.411765,4.058824,5.647059,NaN,NaN,NaN,17.000000,NaN,NaN,106.000000,1.150000,116.473684,10.700000,1.20,174.400000,4.041667,139.600000,14.220000,40.433333
14402,32807,140728,2173-10-27 13:00:00,2173-11-02 13:50:00,NaN,NaN,CORONARY ARTERY DISEASE,0,M,81.0,6.0,16.178082,75.050000,128.531250,69.875000,35.776000,98.287500,3.461538,3.153846,5.615385,18.0,40.0,3.90,22.250000,NaN,NaN,102.500000,1.250000,107.133333,10.840000,1.15,158.000000,4.493750,134.000000,8.380000,52.316667


In [4]:
filename = 'DIAGNOSES_ICD.csv.gz'
with gzip.open(filename, 'rt', encoding='utf-8') as file:
    diag_icd = pd.read_csv(file)
diag_icd

,row_id,subject_id,hadm_id,seq_num,icd9_code
0,1,2,163353,1.0,V3001
1,2,2,163353,2.0,V053
2,3,2,163353,3.0,V290
3,4,3,145834,1.0,0389
4,5,3,145834,2.0,78559
...,...,...,...,...,...
225340,531629,73362,195396,6.0,41401
225341,531630,73362,195396,7.0,V4582
225342,531631,73362,195396,8.0,V4501
225343,531632,73362,195396,9.0,412


In [5]:
diag_icd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225345 entries, 0 to 225344
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   row_id      225345 non-null  int64  
 1   subject_id  225345 non-null  int64  
 2   hadm_id     225345 non-null  int64  
 3   seq_num     225339 non-null  float64
 4   icd9_code   225339 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 8.6+ MB


In [6]:
diag_icd[diag_icd['icd9_code'].isnull()]

,row_id,subject_id,hadm_id,seq_num,icd9_code
86698,126401,11245,148742,NaN,NaN
96418,142003,12607,190435,NaN,NaN
132017,192550,17269,176570,NaN,NaN
136265,198343,17796,142890,NaN,NaN
178085,260938,23407,119146,NaN,NaN
182195,268004,23987,186175,NaN,NaN


In [7]:
diag_icd = diag_icd[diag_icd['icd9_code'].notnull()]
diag_icd

,row_id,subject_id,hadm_id,seq_num,icd9_code
0,1,2,163353,1.0,V3001
1,2,2,163353,2.0,V053
2,3,2,163353,3.0,V290
3,4,3,145834,1.0,0389
4,5,3,145834,2.0,78559
...,...,...,...,...,...
225340,531629,73362,195396,6.0,41401
225341,531630,73362,195396,7.0,V4582
225342,531631,73362,195396,8.0,V4501
225343,531632,73362,195396,9.0,412


In [8]:
# uploading d_icd
d_icd = pd.read_csv("D_ICD_DIAGNOSES.csv.gz")
d_icd

,row_id,icd9_code,short_title,long_title
0,1,01716,Erythem nod tb-oth test,Erythema nodosum with hypersensitivity reactio...
1,2,01720,TB periph lymph-unspec,"Tuberculosis of peripheral lymph nodes, unspec..."
2,3,01721,TB periph lymph-no exam,"Tuberculosis of peripheral lymph nodes, bacter..."
3,4,01722,TB periph lymph-exam unk,"Tuberculosis of peripheral lymph nodes, bacter..."
4,5,01723,TB periph lymph-micro dx,"Tuberculosis of peripheral lymph nodes, tuberc..."
...,...,...,...,...
14562,14563,V8712,Contact/exposure benzene,Contact with and (suspected) exposure to benzene
14563,14564,V8719,Cont/exp haz aromat NEC,Contact with and (suspected) exposure to other...
14564,14565,V872,Cont/exp hazard chem NEC,Contact with and (suspected) exposure to other...
14565,14566,V8731,Contact/exposure mold,Contact with and (suspected) exposure to mold


In [9]:
diag_icd1 = diag_icd.copy()
diag_icd1 = diag_icd1.merge(d_icd[['icd9_code','long_title']], on=['icd9_code'], how='left')
diag_icd1

,row_id,subject_id,hadm_id,seq_num,icd9_code,long_title
0,1,2,163353,1.0,V3001,"Single liveborn, born in hospital, delivered b..."
1,2,2,163353,2.0,V053,Need for prophylactic vaccination and inoculat...
2,3,2,163353,3.0,V290,Observation for suspected infectious condition
3,4,3,145834,1.0,0389,Unspecified septicemia
4,5,3,145834,2.0,78559,Other shock without mention of trauma
...,...,...,...,...,...,...
225334,531629,73362,195396,6.0,41401,Coronary atherosclerosis of native coronary ar...
225335,531630,73362,195396,7.0,V4582,Percutaneous transluminal coronary angioplasty...
225336,531631,73362,195396,8.0,V4501,Cardiac pacemaker in situ
225337,531632,73362,195396,9.0,412,Old myocardial infarction


In [10]:
diag_icd2 = diag_icd1.copy()
diag_icd2.sort_values(['subject_id','hadm_id','seq_num'],inplace=True)
diag_icd2 = (diag_icd2.groupby(['subject_id','hadm_id'])['long_title']
            .apply(lambda x: "; ".join(np.unique(x.dropna().astype(str))))
            .reset_index())
diag_icd2

,subject_id,hadm_id,long_title
0,2,163353,Need for prophylactic vaccination and inoculat...
1,3,145834,"Acute kidney failure, unspecified; Cardiac arr..."
2,5,178980,Need for prophylactic vaccination and inoculat...
3,7,118037,Need for prophylactic vaccination and inoculat...
4,8,159514,Need for prophylactic vaccination and inoculat...
...,...,...,...
26825,32802,128876,Need for prophylactic vaccination and inoculat...
26826,32803,105824,25-26 completed weeks of gestation; Anemia of ...
26827,32806,104049,33-34 completed weeks of gestation; Hemolytic ...
26828,32807,140728,Abdominal aneurysm without mention of rupture;...


In [11]:
diag_icd2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26830 entries, 0 to 26829
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   subject_id  26830 non-null  int64 
 1   hadm_id     26830 non-null  int64 
 2   long_title  26830 non-null  object
dtypes: int64(2), object(1)
memory usage: 629.0+ KB


In [12]:
df_merged0 = df_stru.merge(diag_icd2,on=['subject_id','hadm_id'],how='left')
df_merged0

,subject_id,hadm_id,admittime,dischtime,edregtime,edouttime,diagnosis,hospital_expire_flag,gender,age,hospital_days,Respiratory rate,Heart rate,Systolic blood pressure,Diastolic blood pressure,Body temperature,Saturation of peripheral oxygen,Glasgow coma scale - eye,Glasgow coma scale - verbal,Glasgow coma scale - motor,Alanine aminotransferase,Aspartate aminotransferase,Albumin,Blood urea nitrogen,Bilirubin,C-reactive protein,Chloride,Creatinine,Glucose,Hemoglobin,Prothrombin time (INR),Platelets,Potassium,Sodium,White blood cell,Activated partial thromboplastin time,long_title
0,9,150750,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-09 11:13:00,2149-11-09 13:18:00,HEMORRHAGIC CVA,1,M,41.8,4.9,17.542857,83.771429,133.588235,71.000000,36.096000,98.285714,1.000000,1.000000,1.083333,NaN,NaN,NaN,30.500000,NaN,NaN,102.500000,1.850000,149.000000,12.050000,1.10,318.000000,3.600000,133.400000,11.650000,23.300000,"Acute kidney failure, unspecified; Congestive ..."
1,11,194540,2178-04-16 06:18:00,2178-05-11 19:00:00,2178-04-15 20:46:00,2178-04-16 06:53:00,BRAIN MASS,0,F,50.1,25.5,18.176471,84.918919,106.666667,54.194444,36.768750,96.655172,4.000000,4.941176,5.882353,NaN,NaN,NaN,16.500000,NaN,NaN,106.000000,0.700000,128.333333,12.550000,NaN,250.500000,3.850000,141.500000,11.050000,NaN,Malignant neoplasm of parietal lobe
2,13,143045,2167-01-08 18:43:00,2167-01-15 15:15:00,NaN,NaN,CORONARY ARTERY DISEASE,0,F,39.9,6.9,17.646552,98.017241,107.250000,48.500000,37.122500,98.592920,3.782609,4.304348,5.608696,16.0,16.0,3.90,14.833333,NaN,NaN,107.200000,0.583333,158.833333,10.820000,1.40,135.500000,4.175000,138.375000,17.200000,33.875000,Coronary atherosclerosis of native coronary ar...
3,19,109235,2108-08-05 16:25:00,2108-08-11 11:29:00,2108-08-05 08:01:00,2108-08-05 20:12:00,C-2 FRACTURE,0,M,91.0,5.8,20.583333,98.416667,146.708333,62.000000,36.872857,98.208333,4.000000,4.200000,6.000000,NaN,NaN,NaN,25.000000,NaN,NaN,105.000000,1.200000,139.000000,12.800000,1.30,216.000000,4.200000,138.000000,10.100000,30.400000,Accidental fall on or from other stairs or ste...
4,20,157681,2183-04-28 09:45:00,2183-05-03 14:45:00,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,F,75.9,5.2,15.512821,79.121951,118.333333,42.166667,36.000000,98.435897,3.000000,3.000000,4.333333,NaN,NaN,NaN,18.000000,NaN,NaN,108.000000,0.800000,153.750000,9.500000,1.45,121.500000,4.088889,137.500000,17.500000,32.750000,Coronary atherosclerosis of native coronary ar...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14399,32786,165351,2152-11-15 20:26:00,2152-12-08 11:00:00,2152-11-15 18:39:00,2152-11-15 22:21:00,SUBARACHNOID HEMORRHAGE,0,M,53.7,22.6,22.461314,68.940666,152.684211,55.041509,37.954350,97.714286,3.112676,1.706714,4.616197,67.0,47.5,3.44,34.793103,NaN,NaN,105.275862,0.927586,161.782609,10.844444,1.10,448.740741,4.330303,139.896552,16.648148,26.382609,Acute respiratory failure; Atrial fibrillation...
14400,32786,163011,2152-12-09 19:17:00,2152-12-15 14:30:00,2152-12-09 15:28:00,2152-12-09 22:54:00,ALTERED MENTAL STATUS,0,M,53.8,5.8,23.518519,61.888889,131.423077,66.192308,37.230000,96.923077,2.000000,1.000000,1.857143,475.0,269.5,3.70,82.000000,NaN,NaN,107.000000,1.350000,159.000000,10.800000,1.20,457.000000,4.650000,147.000000,14.150000,25.700000,"Acute kidney failure, unspecified; Candidiasis..."
14401,32787,157308,2102-10-02 07:15:00,2102-10-09 16:42:00,NaN,NaN,AORTIC STENOSIS\AORTIC AND MITRAL VALVE REPLAC...,0,M,44.7,7.4,16.694737,81.526316,115.500000,71.904762,37.500000,97.543478,3.411765,4.058824,5.647059,NaN,NaN,NaN,17.000000,NaN,NaN,106.000000,1.150000,116.473684,10.700000,1.20,174.400000,4.041667,139.600000,14.220000,40.433333,"Accidental cut, puncture, perforation or hemor..."
14402,32807,140728,2173-10-27 13:00:00,2173-11-02 13:50:00,NaN,NaN,CORONARY ARTERY DISEASE,0,M,81.0,6.0,16.178082,75.050000,128.531250,69.875000,35.776000,98.287500,3.4615

In [13]:
df_merged0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14404 entries, 0 to 14403
Data columns (total 37 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   subject_id                             14404 non-null  int64  
 1   hadm_id                                14404 non-null  int64  
 2   admittime                              14404 non-null  object 
 3   dischtime                              14404 non-null  object 
 4   edregtime                              7279 non-null   object 
 5   edouttime                              7279 non-null   object 
 6   diagnosis                              14404 non-null  object 
 7   hospital_expire_flag                   14404 non-null  int64  
 8   gender                                 14404 non-null  object 
 9   age                                    14404 non-null  float64
 10  hospital_days                          14404 non-null  float64
 11  Re

# Add Chief Complaint

In [14]:
filename = 'NOTEEVENTS.csv.gz'
with gzip.open(filename, 'rt', encoding='utf-8') as file:
    df_event = pd.read_csv(file)
df_event

C:\Users\AaronChen\AppData\Local\Temp\ipykernel_16188\3632815017.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_event = pd.read_csv(file)


,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,text
0,738407,20409,NaN,2119-01-04 00:00:00,2119-01-04 12:59:00,NaN,Radiology,ABDOMEN U.S. (COMPLETE STUDY),NaN,NaN,[**2119-1-4**] 12:59 PM\n ABDOMEN U.S. (COMPLE...
1,738408,20409,NaN,2119-01-09 00:00:00,2119-01-09 13:05:00,NaN,Radiology,MR LIVER WITH CONTRAST,NaN,NaN,[**2119-1-9**] 1:05 PM\n MR LIVER WITH CONTRAS...
2,738409,20409,NaN,2119-01-16 00:00:00,2119-01-16 21:24:00,NaN,Radiology,CHEST (PORTABLE AP),NaN,NaN,[**2119-1-16**] 9:24 PM\n CHEST (PORTABLE AP) ...
3,738410,20409,NaN,2119-01-18 00:00:00,2119-01-18 13:24:00,NaN,Radiology,CT ABD W&W/O C,NaN,NaN,[**2119-1-18**] 1:24 PM\n CT ABD W&W/O C; CT P...
4,738411,20409,NaN,2119-01-18 00:00:00,2119-01-18 15:45:00,NaN,Radiology,PARACENTESIS DIAG. OR THERAPEUTIC,NaN,NaN,[**2119-1-18**] 3:45 PM\n PARACENTESIS DIAG. O...
...,...,...,...,...,...,...,...,...,...,...,...
880102,2083175,32806,104049.0,2162-11-02 00:00:00,2162-11-02 13:09:00,2162-11-02 13:27:00,Nursing/other,Report,14461.0,NaN,1. FEN\nTF= min140cc/k/d of BM/E24. min47cc q4...
880103,2083176,32806,104049.0,2162-11-02 00:00:00,2162-11-02 13:09:00,2162-11-02 16:36:00,Nursing/other,Report,16865.0,NaN,I have examined pt. & agree w/ [**First Name8 ...
880104,2083177,32806,104049.0,2162-11-03 00:00:00,2162-11-03 01:57:00,2162-11-03 02:13:00,Nursing/other,Report,17604.0,NaN,NPN nights\n\n\nFluids/Nutrition: Weight 2025...
880105,2083178,32806,104049.0,2162-11-03 00:00:00,2162-11-03 09:55:00,2162-11-03 09:59:00,Nursing/other,Report,21222.0,NaN,Attending Note\nDay of life 12 PMA 35 [**2-7**...


In [15]:
df_event1 = df_event.copy()
df_event1 = df_event1[['subject_id','hadm_id','text']]
df_event1 = df_event1.dropna(subset=['hadm_id'],inplace=False)
df_event1 = (df_event1.groupby(['subject_id','hadm_id'])['text']
            .apply(lambda x: "; ".join(np.unique(x.dropna().astype(str))))
            .reset_index())
df_event1['textLength'] = df_event1['text'].apply(lambda x: len(x))
df_event1 = df_event1.sort_values(by=['textLength'], ascending=True)
df_event1 = df_event1[df_event1['hadm_id'].isin(df_merged0['hadm_id'])]
df_event1

,subject_id,hadm_id,text,textLength
13206,14597,163084.0,NPN (NOC): PLEASE SEE FHPA AND FLOW SHEET FOR...,140
22357,24703,134412.0,Resp Care Note:\n\nPt emergently intub #8.0 @ ...,196
18623,20481,197254.0,"Pt with stable vital signs, a+ox3 amb with ste...",197
12065,13319,113674.0,Resp Care Note:\n\nPt intub with OETT in cath ...,255
20164,22204,160986.0,"OR admission: pt. admitted intubated, sedated ...",348
...,...,...,...,...
15920,17497,166377.0,48 YR OLD C PUL FIBROSIS C FAILURE TO WEAN FRO...,293359
18114,19911,180900.0,(Continued)\nfor evidence of increased bleedin...,300780
240,286,106909.0,7p-7a addendum\nLasix 80mg IVP given at 0500 d...,314622
16682,18318,128140.0,"(Continued)\n area pink this am, better this a...",330710


In [16]:
df_event1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14358 entries, 13206 to 14238
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   subject_id  14358 non-null  int64  
 1   hadm_id     14358 non-null  float64
 2   text        14358 non-null  object 
 3   textLength  14358 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 560.9+ KB


In [17]:
device = 0 if torch.cuda.is_available() else -1

In [18]:
model_name = "biomedical-ner-all"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

nlp = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=device)

text_max = 5000

max_length = 500

D:\anaconda\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [19]:
def CC_extraction(text, text_type):

    def chunk_text(text, tokenizer, max_length):
        tokens = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=text_max)
        input_ids = tokens['input_ids'][0].tolist()
        chunks = []
        for i in range(0, len(input_ids), max_length - 2): 
            chunk = input_ids[i:i + max_length - 2]
            chunk = [tokenizer.cls_token_id] + chunk + [tokenizer.sep_token_id]
            chunks.append(chunk)
        return chunks

    chunks = chunk_text(text, tokenizer, max_length)

    entities = []
    for chunk in chunks:
        chunk_text = tokenizer.decode(chunk, skip_special_tokens=True)
        entities.extend(nlp(chunk_text))

    entity_labels = {
        "Sign_symptom": "status",
        "Detailed_description": "details",
        "Severity": "severity",
        "Medication": "medications"
    }

    results = {label: [] for label in entity_labels.values()}

    current_entity = ""
    current_label = ""
    for entity in entities:
        entity_text = entity['word'].replace('##', '')
        entity_label = entity['entity_group']

        if entity_label in entity_labels:
            if entity_label == current_label:
                if entity['word'].startswith('##'):
                    current_entity += entity_text
                else:
                    current_entity += ' ' + entity_text
            else:
                if current_label and current_entity:
                    results[entity_labels[current_label]].append(current_entity.strip())
                current_entity = entity_text
                current_label = entity_label

    if current_label and current_entity and current_label in entity_labels:
        results[entity_labels[current_label]].append(current_entity.strip())

    message = ', '.join(results[text_type])

    return message

In [20]:
df_event2 = df_event1.copy()
df_event2['Chief Complaint'] = df_event2['text'].apply(lambda x: CC_extraction(x, "status"))
df_event2

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


,subject_id,hadm_id,text,textLength,Chief Complaint
13206,14597,163084.0,NPN (NOC): PLEASE SEE FHPA AND FLOW SHEET FOR...,140,
22357,24703,134412.0,Resp Care Note:\n\nPt emergently intub #8.0 @ ...,196,
18623,20481,197254.0,"Pt with stable vital signs, a+ox3 amb with ste...",197,
12065,13319,113674.0,Resp Care Note:\n\nPt intub with OETT in cath ...,255,
20164,22204,160986.0,"OR admission: pt. admitted intubated, sedated ...",348,extremeties cool mo
...,...,...,...,...,...
15920,17497,166377.0,48 YR OLD C PUL FIBROSIS C FAILURE TO WEAN FRO...,293359,"pul fibrosis c, fatigued neuro lethargic fatig..."
18114,19911,180900.0,(Continued)\nfor evidence of increased bleedin...,300780,"residual, fluid, alert movement limbs, rest, r..."
240,286,106909.0,7p-7a addendum\nLasix 80mg IVP given at 0500 d...,314622,"wedge, pivot, diuresis tears tem edema, hyperg..."
16682,18318,128140.0,"(Continued)\n area pink this am, better this a...",330710,"abrasion, lete dozing lift fall, cough cough u..."


In [21]:
df_merged1 = df_merged0.copy()
df_merged1 = df_merged1.merge(df_event2[['subject_id','hadm_id','Chief Complaint']], 
                              on=['subject_id','hadm_id'],
                              how='left')
df_merged1

,subject_id,hadm_id,admittime,dischtime,edregtime,edouttime,diagnosis,hospital_expire_flag,gender,age,hospital_days,Respiratory rate,Heart rate,Systolic blood pressure,Diastolic blood pressure,Body temperature,Saturation of peripheral oxygen,Glasgow coma scale - eye,Glasgow coma scale - verbal,Glasgow coma scale - motor,Alanine aminotransferase,Aspartate aminotransferase,Albumin,Blood urea nitrogen,Bilirubin,C-reactive protein,Chloride,Creatinine,Glucose,Hemoglobin,Prothrombin time (INR),Platelets,Potassium,Sodium,White blood cell,Activated partial thromboplastin time,long_title,Chief Complaint
0,9,150750,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-09 11:13:00,2149-11-09 13:18:00,HEMORRHAGIC CVA,1,M,41.8,4.9,17.542857,83.771429,133.588235,71.000000,36.096000,98.285714,1.000000,1.000000,1.083333,NaN,NaN,NaN,30.500000,NaN,NaN,102.500000,1.850000,149.000000,12.050000,1.10,318.000000,3.600000,133.400000,11.650000,23.300000,"Acute kidney failure, unspecified; Congestive ...","awake, pain fl, ec, coughing vomited, vomit, e..."
1,11,194540,2178-04-16 06:18:00,2178-05-11 19:00:00,2178-04-15 20:46:00,2178-04-16 06:53:00,BRAIN MASS,0,F,50.1,25.5,18.176471,84.918919,106.666667,54.194444,36.768750,96.655172,4.000000,4.941176,5.882353,NaN,NaN,NaN,16.500000,NaN,NaN,106.000000,0.700000,128.333333,12.550000,NaN,250.500000,3.850000,141.500000,11.050000,NaN,Malignant neoplasm of parietal lobe,"meta, meta meta lift hold discomfort b, masses..."
2,13,143045,2167-01-08 18:43:00,2167-01-15 15:15:00,NaN,NaN,CORONARY ARTERY DISEASE,0,F,39.9,6.9,17.646552,98.017241,107.250000,48.500000,37.122500,98.592920,3.782609,4.304348,5.608696,16.0,16.0,3.90,14.833333,NaN,NaN,107.200000,0.583333,158.833333,10.820000,1.40,135.500000,4.175000,138.375000,17.200000,33.875000,Coronary atherosclerosis of native coronary ar...,"anxiety pain pain, diminished breath collapse ..."
3,19,109235,2108-08-05 16:25:00,2108-08-11 11:29:00,2108-08-05 08:01:00,2108-08-05 20:12:00,C-2 FRACTURE,0,M,91.0,5.8,20.583333,98.416667,146.708333,62.000000,36.872857,98.208333,4.000000,4.200000,6.000000,NaN,NaN,NaN,25.000000,NaN,NaN,105.000000,1.200000,139.000000,12.800000,1.30,216.000000,4.200000,138.000000,10.100000,30.400000,Accidental fall on or from other stairs or ste...,"ur, pain, ub cat, abrasions, at mass, trauma, ..."
4,20,157681,2183-04-28 09:45:00,2183-05-03 14:45:00,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,F,75.9,5.2,15.512821,79.121951,118.333333,42.166667,36.000000,98.435897,3.000000,3.000000,4.333333,NaN,NaN,NaN,18.000000,NaN,NaN,108.000000,0.800000,153.750000,9.500000,1.45,121.500000,4.088889,137.500000,17.500000,32.750000,Coronary atherosclerosis of native coronary ar...,"lete, apnea, drift back to sleep, obe, hypovol..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14399,32786,165351,2152-11-15 20:26:00,2152-12-08 11:00:00,2152-11-15 18:39:00,2152-11-15 22:21:00,SUBARACHNOID HEMORRHAGE,0,M,53.7,22.6,22.461314,68.940666,152.684211,55.041509,37.954350,97.714286,3.112676,1.706714,4.616197,67.0,47.5,3.44,34.793103,NaN,NaN,105.275862,0.927586,161.782609,10.844444,1.10,448.740741,4.330303,139.896552,16.648148,26.382609,Acute respiratory failure; Atrial fibrillation...,"hypertension, withdraw pain grim weakness spon..."
14400,32786,163011,2152-12-09 19:17:00,2152-12-15 14:30:00,2152-12-09 15:28:00,2152-12-09 22:54:00,ALTERED MENTAL STATUS,0,M,53.8,5.8,23.518519,61.888889,131.423077,66.192308,37.230000,96.923077,2.000000,1.000000,1.857143,475.0,269.5,3.70,82.000000,NaN,NaN,107.000000,1.350000,159.000000,10.800000,1.20,457.000000,4.650000,147.000000,14.150000,25.700000,"Acute kidney failure, unspecified; Candidiasis...","gall altered mental l gall adenomatosis, duct ..."
14401,32787,157308,2102-10-02 07:15:00,2102-10-09 16:42:00,NaN,NaN,AORTIC STENOSIS\AORTIC AND MITRAL VALVE REPLAC...,0,M,44.7,7.4,16.694737,81.526316,115.500000,71.904762,37.500000,97.

In [24]:
df_1 = df_merged1.copy()

cols = ["subject_id","hadm_id","age"] + df_1.columns.tolist()[11:]
cols.append('hospital_expire_flag')
df_1 = df_1[cols]
df_1

,subject_id,hadm_id,age,Respiratory rate,Heart rate,Systolic blood pressure,Diastolic blood pressure,Body temperature,Saturation of peripheral oxygen,Glasgow coma scale - eye,Glasgow coma scale - verbal,Glasgow coma scale - motor,Alanine aminotransferase,Aspartate aminotransferase,Albumin,Blood urea nitrogen,Bilirubin,C-reactive protein,Chloride,Creatinine,Glucose,Hemoglobin,Prothrombin time (INR),Platelets,Potassium,Sodium,White blood cell,Activated partial thromboplastin time,long_title,Chief Complaint,hospital_expire_flag
0,9,150750,41.8,17.542857,83.771429,133.588235,71.000000,36.096000,98.285714,1.000000,1.000000,1.083333,NaN,NaN,NaN,30.500000,NaN,NaN,102.500000,1.850000,149.000000,12.050000,1.10,318.000000,3.600000,133.400000,11.650000,23.300000,"Acute kidney failure, unspecified; Congestive ...","awake, pain fl, ec, coughing vomited, vomit, e...",1
1,11,194540,50.1,18.176471,84.918919,106.666667,54.194444,36.768750,96.655172,4.000000,4.941176,5.882353,NaN,NaN,NaN,16.500000,NaN,NaN,106.000000,0.700000,128.333333,12.550000,NaN,250.500000,3.850000,141.500000,11.050000,NaN,Malignant neoplasm of parietal lobe,"meta, meta meta lift hold discomfort b, masses...",0
2,13,143045,39.9,17.646552,98.017241,107.250000,48.500000,37.122500,98.592920,3.782609,4.304348,5.608696,16.0,16.0,3.90,14.833333,NaN,NaN,107.200000,0.583333,158.833333,10.820000,1.40,135.500000,4.175000,138.375000,17.200000,33.875000,Coronary atherosclerosis of native coronary ar...,"anxiety pain pain, diminished breath collapse ...",0
3,19,109235,91.0,20.583333,98.416667,146.708333,62.000000,36.872857,98.208333,4.000000,4.200000,6.000000,NaN,NaN,NaN,25.000000,NaN,NaN,105.000000,1.200000,139.000000,12.800000,1.30,216.000000,4.200000,138.000000,10.100000,30.400000,Accidental fall on or from other stairs or ste...,"ur, pain, ub cat, abrasions, at mass, trauma, ...",0
4,20,157681,75.9,15.512821,79.121951,118.333333,42.166667,36.000000,98.435897,3.000000,3.000000,4.333333,NaN,NaN,NaN,18.000000,NaN,NaN,108.000000,0.800000,153.750000,9.500000,1.45,121.500000,4.088889,137.500000,17.500000,32.750000,Coronary atherosclerosis of native coronary ar...,"lete, apnea, drift back to sleep, obe, hypovol...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14399,32786,165351,53.7,22.461314,68.940666,152.684211,55.041509,37.954350,97.714286,3.112676,1.706714,4.616197,67.0,47.5,3.44,34.793103,NaN,NaN,105.275862,0.927586,161.782609,10.844444,1.10,448.740741,4.330303,139.896552,16.648148,26.382609,Acute respiratory failure; Atrial fibrillation...,"hypertension, withdraw pain grim weakness spon...",0
14400,32786,163011,53.8,23.518519,61.888889,131.423077,66.192308,37.230000,96.923077,2.000000,1.000000,1.857143,475.0,269.5,3.70,82.000000,NaN,NaN,107.000000,1.350000,159.000000,10.800000,1.20,457.000000,4.650000,147.000000,14.150000,25.700000,"Acute kidney failure, unspecified; Candidiasis...","gall altered mental l gall adenomatosis, duct ...",0
14401,32787,157308,44.7,16.694737,81.526316,115.500000,71.904762,37.500000,97.543478,3.411765,4.058824,5.647059,NaN,NaN,NaN,17.000000,NaN,NaN,106.000000,1.150000,116.473684,10.700000,1.20,174.400000,4.041667,139.600000,14.220000,40.433333,"Accidental cut, puncture, perforation or hemor...","sl restless anxiety bs rash letgic, agitated a...",0
14402,32807,140728,81.0,16.178082,75.050000,128.531250,69.875000,35.776000,98.287500,3.461538,3.153846,5.615385,18.0,40.0,3.90,22.250000,NaN,NaN,102.500000,1.250000,107.133333,10.840000,1.15,158.000000,4.493750,134.000000,8.380000,52.316667,Abdominal aneurysm without mention of rupture;...,"pain ecy, ed, ested cough, utum, soft void uri...",0


In [25]:
dict_transfor = {
'age': 'Age (year)',
'Respiratory rate': 'Respiratory rate (min)',
'Heart rate': 'Heart rate (min)',
'Systolic blood pressure': 'SBP (mmHg)',
'Diastolic blood pressure': 'DBP (mmHg)',
'Body temperature': 'Body temperature (℃)',
'Saturation of peripheral oxygen': 'SpO2 (%)',
'Glasgow coma scale - eye': 'GCS–eye',
'Glasgow coma scale - verbal': 'GCS–verbal',
'Glasgow coma scale - motor': 'GCS–motor',
'Alanine aminotransferase': 'ALT (Units/L)',
'Aspartate aminotransferase': 'AST (Units/L)',
'Albumin': 'Albumin (g/dL)',
'Blood urea nitrogen': 'BUN (mg/dL)',
'Bilirubin': 'Bilirubin (mg/dL)',
'C-reactive protein': 'CRP (mg/dL)',
'Chloride': 'Chloride (mmol/L)',
'Creatinine': 'Creatinine (mg/dL)',
'Glucose': 'Glucose (mg/dL)',
'Hemoglobin': 'Hemoglobin (g/dL)',
'Prothrombin time (INR)': 'Prothrombin time (INR)',
'Platelets': 'Platelets (10^3/µL)',
'Potassium': 'Potassium (mmol/L)',
'Sodium': 'Sodium (mmol/L)',
'White blood cell': 'WBC (10^3/µL)',
'Activated partial thromboplastin time': 'aPTT (sec)',
'icd_title':'ICD text',
'Chief Complaint':'Chief complaint',
'hospital_expire_flag': 'Label'
}
df_2 = df_1.rename(columns=dict_transfor)
df_2

,subject_id,hadm_id,Age (year),Respiratory rate (min),Heart rate (min),SBP (mmHg),DBP (mmHg),Body temperature (℃),SpO2 (%),GCS–eye,GCS–verbal,GCS–motor,ALT (Units/L),AST (Units/L),Albumin (g/dL),BUN (mg/dL),Bilirubin (mg/dL),CRP (mg/dL),Chloride (mmol/L),Creatinine (mg/dL),Glucose (mg/dL),Hemoglobin (g/dL),Prothrombin time (INR),Platelets (10^3/µL),Potassium (mmol/L),Sodium (mmol/L),WBC (10^3/µL),aPTT (sec),long_title,Chief complaint,Label
0,9,150750,41.8,17.542857,83.771429,133.588235,71.000000,36.096000,98.285714,1.000000,1.000000,1.083333,NaN,NaN,NaN,30.500000,NaN,NaN,102.500000,1.850000,149.000000,12.050000,1.10,318.000000,3.600000,133.400000,11.650000,23.300000,"Acute kidney failure, unspecified; Congestive ...","awake, pain fl, ec, coughing vomited, vomit, e...",1
1,11,194540,50.1,18.176471,84.918919,106.666667,54.194444,36.768750,96.655172,4.000000,4.941176,5.882353,NaN,NaN,NaN,16.500000,NaN,NaN,106.000000,0.700000,128.333333,12.550000,NaN,250.500000,3.850000,141.500000,11.050000,NaN,Malignant neoplasm of parietal lobe,"meta, meta meta lift hold discomfort b, masses...",0
2,13,143045,39.9,17.646552,98.017241,107.250000,48.500000,37.122500,98.592920,3.782609,4.304348,5.608696,16.0,16.0,3.90,14.833333,NaN,NaN,107.200000,0.583333,158.833333,10.820000,1.40,135.500000,4.175000,138.375000,17.200000,33.875000,Coronary atherosclerosis of native coronary ar...,"anxiety pain pain, diminished breath collapse ...",0
3,19,109235,91.0,20.583333,98.416667,146.708333,62.000000,36.872857,98.208333,4.000000,4.200000,6.000000,NaN,NaN,NaN,25.000000,NaN,NaN,105.000000,1.200000,139.000000,12.800000,1.30,216.000000,4.200000,138.000000,10.100000,30.400000,Accidental fall on or from other stairs or ste...,"ur, pain, ub cat, abrasions, at mass, trauma, ...",0
4,20,157681,75.9,15.512821,79.121951,118.333333,42.166667,36.000000,98.435897,3.000000,3.000000,4.333333,NaN,NaN,NaN,18.000000,NaN,NaN,108.000000,0.800000,153.750000,9.500000,1.45,121.500000,4.088889,137.500000,17.500000,32.750000,Coronary atherosclerosis of native coronary ar...,"lete, apnea, drift back to sleep, obe, hypovol...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14399,32786,165351,53.7,22.461314,68.940666,152.684211,55.041509,37.954350,97.714286,3.112676,1.706714,4.616197,67.0,47.5,3.44,34.793103,NaN,NaN,105.275862,0.927586,161.782609,10.844444,1.10,448.740741,4.330303,139.896552,16.648148,26.382609,Acute respiratory failure; Atrial fibrillation...,"hypertension, withdraw pain grim weakness spon...",0
14400,32786,163011,53.8,23.518519,61.888889,131.423077,66.192308,37.230000,96.923077,2.000000,1.000000,1.857143,475.0,269.5,3.70,82.000000,NaN,NaN,107.000000,1.350000,159.000000,10.800000,1.20,457.000000,4.650000,147.000000,14.150000,25.700000,"Acute kidney failure, unspecified; Candidiasis...","gall altered mental l gall adenomatosis, duct ...",0
14401,32787,157308,44.7,16.694737,81.526316,115.500000,71.904762,37.500000,97.543478,3.411765,4.058824,5.647059,NaN,NaN,NaN,17.000000,NaN,NaN,106.000000,1.150000,116.473684,10.700000,1.20,174.400000,4.041667,139.600000,14.220000,40.433333,"Accidental cut, puncture, perforation or hemor...","sl restless anxiety bs rash letgic, agitated a...",0
14402,32807,140728,81.0,16.178082,75.050000,128.531250,69.875000,35.776000,98.287500,3.461538,3.153846,5.615385,18.0,40.0,3.90,22.250000,NaN,NaN,102.500000,1.250000,107.133333,10.840000,1.15,158.000000,4.493750,134.000000,8.380000,52.316667,Abdominal aneurysm without mention of rupture;...,"pain ecy, ed, ested cough, utum, soft void uri...",0


In [26]:
df_2.to_csv("journalCC_MIMIC3_withID.csv",index=False)

In [77]:
df_merged2 = df_merged1.copy()
cols = ["age"] + df_merged2.columns.tolist()[11:]
cols.append('hospital_expire_flag')
df_merged2 = df_merged2[cols]
df_merged2

,age,Respiratory rate,Heart rate,Systolic blood pressure,Diastolic blood pressure,Body temperature,Saturation of peripheral oxygen,Glasgow coma scale - eye,Glasgow coma scale - verbal,Glasgow coma scale - motor,Alanine aminotransferase,Aspartate aminotransferase,Albumin,Blood urea nitrogen,Bilirubin,C-reactive protein,Chloride,Creatinine,Glucose,Hemoglobin,Prothrombin time (INR),Platelets,Potassium,Sodium,White blood cell,Activated partial thromboplastin time,long_title,Chief Complaint,hospital_expire_flag
0,41.8,17.542857,83.771429,133.588235,71.000000,36.096000,98.285714,1.000000,1.000000,1.083333,NaN,NaN,NaN,30.500000,NaN,NaN,102.500000,1.850000,149.000000,12.050000,1.10,318.000000,3.600000,133.400000,11.650000,23.300000,"Acute kidney failure, unspecified; Congestive ...","awake, pain fl, ec, coughing vomited, vomit, e...",1
1,50.1,18.176471,84.918919,106.666667,54.194444,36.768750,96.655172,4.000000,4.941176,5.882353,NaN,NaN,NaN,16.500000,NaN,NaN,106.000000,0.700000,128.333333,12.550000,NaN,250.500000,3.850000,141.500000,11.050000,NaN,Malignant neoplasm of parietal lobe,"meta, meta meta lift hold discomfort b, masses...",0
2,39.9,17.646552,98.017241,107.250000,48.500000,37.122500,98.592920,3.782609,4.304348,5.608696,16.0,16.0,3.90,14.833333,NaN,NaN,107.200000,0.583333,158.833333,10.820000,1.40,135.500000,4.175000,138.375000,17.200000,33.875000,Coronary atherosclerosis of native coronary ar...,"anxiety pain pain, diminished breath collapse ...",0
3,91.0,20.583333,98.416667,146.708333,62.000000,36.872857,98.208333,4.000000,4.200000,6.000000,NaN,NaN,NaN,25.000000,NaN,NaN,105.000000,1.200000,139.000000,12.800000,1.30,216.000000,4.200000,138.000000,10.100000,30.400000,Accidental fall on or from other stairs or ste...,"ur, pain, ub cat, abrasions, at mass, trauma, ...",0
4,75.9,15.512821,79.121951,118.333333,42.166667,36.000000,98.435897,3.000000,3.000000,4.333333,NaN,NaN,NaN,18.000000,NaN,NaN,108.000000,0.800000,153.750000,9.500000,1.45,121.500000,4.088889,137.500000,17.500000,32.750000,Coronary atherosclerosis of native coronary ar...,"lete, apnea, drift back to sleep, obe, hypovol...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14399,53.7,22.461314,68.940666,152.684211,55.041509,37.954350,97.714286,3.112676,1.706714,4.616197,67.0,47.5,3.44,34.793103,NaN,NaN,105.275862,0.927586,161.782609,10.844444,1.10,448.740741,4.330303,139.896552,16.648148,26.382609,Acute respiratory failure; Atrial fibrillation...,"hypertension, withdraw pain grim weakness spon...",0
14400,53.8,23.518519,61.888889,131.423077,66.192308,37.230000,96.923077,2.000000,1.000000,1.857143,475.0,269.5,3.70,82.000000,NaN,NaN,107.000000,1.350000,159.000000,10.800000,1.20,457.000000,4.650000,147.000000,14.150000,25.700000,"Acute kidney failure, unspecified; Candidiasis...","gall altered mental l gall adenomatosis, duct ...",0
14401,44.7,16.694737,81.526316,115.500000,71.904762,37.500000,97.543478,3.411765,4.058824,5.647059,NaN,NaN,NaN,17.000000,NaN,NaN,106.000000,1.150000,116.473684,10.700000,1.20,174.400000,4.041667,139.600000,14.220000,40.433333,"Accidental cut, puncture, perforation or hemor...","sl restless anxiety bs rash letgic, agitated a...",0
14402,81.0,16.178082,75.050000,128.531250,69.875000,35.776000,98.287500,3.461538,3.153846,5.615385,18.0,40.0,3.90,22.250000,NaN,NaN,102.500000,1.250000,107.133333,10.840000,1.15,158.000000,4.493750,134.000000,8.380000,52.316667,Abdominal aneurysm without mention of rupture;...,"pain ecy, ed, ested cough, utum, soft void uri...",0


In [78]:
dict_transfor = {
'age': 'Age (year)',
'Respiratory rate': 'Respiratory rate (min)',
'Heart rate': 'Heart rate (min)',
'Systolic blood pressure': 'SBP (mmHg)',
'Diastolic blood pressure': 'DBP (mmHg)',
'Body temperature': 'Body temperature (℃)',
'Saturation of peripheral oxygen': 'SpO2 (%)',
'Glasgow coma scale - eye': 'GCS–eye',
'Glasgow coma scale - verbal': 'GCS–verbal',
'Glasgow coma scale - motor': 'GCS–motor',
'Alanine aminotransferase': 'ALT (Units/L)',
'Aspartate aminotransferase': 'AST (Units/L)',
'Albumin': 'Albumin (g/dL)',
'Blood urea nitrogen': 'BUN (mg/dL)',
'Bilirubin': 'Bilirubin (mg/dL)',
'C-reactive protein': 'CRP (mg/dL)',
'Chloride': 'Chloride (mmol/L)',
'Creatinine': 'Creatinine (mg/dL)',
'Glucose': 'Glucose (mg/dL)',
'Hemoglobin': 'Hemoglobin (g/dL)',
'Prothrombin time (INR)': 'Prothrombin time (INR)',
'Platelets': 'Platelets (10^3/µL)',
'Potassium': 'Potassium (mmol/L)',
'Sodium': 'Sodium (mmol/L)',
'White blood cell': 'WBC (10^3/µL)',
'Activated partial thromboplastin time': 'aPTT (sec)',
'icd_title':'ICD text',
'Chief Complaint':'Chief complaint',
'hospital_expire_flag': 'Label'
}
df_merged3 = df_merged2.rename(columns=dict_transfor)
df_merged3

,Age (year),Respiratory rate (min),Heart rate (min),SBP (mmHg),DBP (mmHg),Body temperature (℃),SpO2 (%),GCS–eye,GCS–verbal,GCS–motor,ALT (Units/L),AST (Units/L),Albumin (g/dL),BUN (mg/dL),Bilirubin (mg/dL),CRP (mg/dL),Chloride (mmol/L),Creatinine (mg/dL),Glucose (mg/dL),Hemoglobin (g/dL),Prothrombin time (INR),Platelets (10^3/µL),Potassium (mmol/L),Sodium (mmol/L),WBC (10^3/µL),aPTT (sec),long_title,Chief complaint,Label
0,41.8,17.542857,83.771429,133.588235,71.000000,36.096000,98.285714,1.000000,1.000000,1.083333,NaN,NaN,NaN,30.500000,NaN,NaN,102.500000,1.850000,149.000000,12.050000,1.10,318.000000,3.600000,133.400000,11.650000,23.300000,"Acute kidney failure, unspecified; Congestive ...","awake, pain fl, ec, coughing vomited, vomit, e...",1
1,50.1,18.176471,84.918919,106.666667,54.194444,36.768750,96.655172,4.000000,4.941176,5.882353,NaN,NaN,NaN,16.500000,NaN,NaN,106.000000,0.700000,128.333333,12.550000,NaN,250.500000,3.850000,141.500000,11.050000,NaN,Malignant neoplasm of parietal lobe,"meta, meta meta lift hold discomfort b, masses...",0
2,39.9,17.646552,98.017241,107.250000,48.500000,37.122500,98.592920,3.782609,4.304348,5.608696,16.0,16.0,3.90,14.833333,NaN,NaN,107.200000,0.583333,158.833333,10.820000,1.40,135.500000,4.175000,138.375000,17.200000,33.875000,Coronary atherosclerosis of native coronary ar...,"anxiety pain pain, diminished breath collapse ...",0
3,91.0,20.583333,98.416667,146.708333,62.000000,36.872857,98.208333,4.000000,4.200000,6.000000,NaN,NaN,NaN,25.000000,NaN,NaN,105.000000,1.200000,139.000000,12.800000,1.30,216.000000,4.200000,138.000000,10.100000,30.400000,Accidental fall on or from other stairs or ste...,"ur, pain, ub cat, abrasions, at mass, trauma, ...",0
4,75.9,15.512821,79.121951,118.333333,42.166667,36.000000,98.435897,3.000000,3.000000,4.333333,NaN,NaN,NaN,18.000000,NaN,NaN,108.000000,0.800000,153.750000,9.500000,1.45,121.500000,4.088889,137.500000,17.500000,32.750000,Coronary atherosclerosis of native coronary ar...,"lete, apnea, drift back to sleep, obe, hypovol...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14399,53.7,22.461314,68.940666,152.684211,55.041509,37.954350,97.714286,3.112676,1.706714,4.616197,67.0,47.5,3.44,34.793103,NaN,NaN,105.275862,0.927586,161.782609,10.844444,1.10,448.740741,4.330303,139.896552,16.648148,26.382609,Acute respiratory failure; Atrial fibrillation...,"hypertension, withdraw pain grim weakness spon...",0
14400,53.8,23.518519,61.888889,131.423077,66.192308,37.230000,96.923077,2.000000,1.000000,1.857143,475.0,269.5,3.70,82.000000,NaN,NaN,107.000000,1.350000,159.000000,10.800000,1.20,457.000000,4.650000,147.000000,14.150000,25.700000,"Acute kidney failure, unspecified; Candidiasis...","gall altered mental l gall adenomatosis, duct ...",0
14401,44.7,16.694737,81.526316,115.500000,71.904762,37.500000,97.543478,3.411765,4.058824,5.647059,NaN,NaN,NaN,17.000000,NaN,NaN,106.000000,1.150000,116.473684,10.700000,1.20,174.400000,4.041667,139.600000,14.220000,40.433333,"Accidental cut, puncture, perforation or hemor...","sl restless anxiety bs rash letgic, agitated a...",0
14402,81.0,16.178082,75.050000,128.531250,69.875000,35.776000,98.287500,3.461538,3.153846,5.615385,18.0,40.0,3.90,22.250000,NaN,NaN,102.500000,1.250000,107.133333,10.840000,1.15,158.000000,4.493750,134.000000,8.380000,52.316667,Abdominal aneurysm without mention of rupture;...,"pain ecy, ed, ested cough, utum, soft void uri...",0


In [79]:
df_merged3.to_csv("journalCC_MIMIC3_subset_StruWithNonStru_renamed.csv",index=False)

In [22]:
df_merged4 = df_merged1.copy()
cols = ["age"] + df_merged4.columns.tolist()[11:] + ["hospital_days"]
cols.append('hospital_expire_flag')
df_merged4 = df_merged4[cols]
df_merged4

,age,Respiratory rate,Heart rate,Systolic blood pressure,Diastolic blood pressure,Body temperature,Saturation of peripheral oxygen,Glasgow coma scale - eye,Glasgow coma scale - verbal,Glasgow coma scale - motor,Alanine aminotransferase,Aspartate aminotransferase,Albumin,Blood urea nitrogen,Bilirubin,C-reactive protein,Chloride,Creatinine,Glucose,Hemoglobin,Prothrombin time (INR),Platelets,Potassium,Sodium,White blood cell,Activated partial thromboplastin time,long_title,Chief Complaint,hospital_days,hospital_expire_flag
0,41.8,17.542857,83.771429,133.588235,71.000000,36.096000,98.285714,1.000000,1.000000,1.083333,NaN,NaN,NaN,30.500000,NaN,NaN,102.500000,1.850000,149.000000,12.050000,1.10,318.000000,3.600000,133.400000,11.650000,23.300000,"Acute kidney failure, unspecified; Congestive ...","awake, pain fl, ec, coughing vomited, vomit, e...",4.9,1
1,50.1,18.176471,84.918919,106.666667,54.194444,36.768750,96.655172,4.000000,4.941176,5.882353,NaN,NaN,NaN,16.500000,NaN,NaN,106.000000,0.700000,128.333333,12.550000,NaN,250.500000,3.850000,141.500000,11.050000,NaN,Malignant neoplasm of parietal lobe,"meta, meta meta lift hold discomfort b, masses...",25.5,0
2,39.9,17.646552,98.017241,107.250000,48.500000,37.122500,98.592920,3.782609,4.304348,5.608696,16.0,16.0,3.90,14.833333,NaN,NaN,107.200000,0.583333,158.833333,10.820000,1.40,135.500000,4.175000,138.375000,17.200000,33.875000,Coronary atherosclerosis of native coronary ar...,"anxiety pain pain, diminished breath collapse ...",6.9,0
3,91.0,20.583333,98.416667,146.708333,62.000000,36.872857,98.208333,4.000000,4.200000,6.000000,NaN,NaN,NaN,25.000000,NaN,NaN,105.000000,1.200000,139.000000,12.800000,1.30,216.000000,4.200000,138.000000,10.100000,30.400000,Accidental fall on or from other stairs or ste...,"ur, pain, ub cat, abrasions, at mass, trauma, ...",5.8,0
4,75.9,15.512821,79.121951,118.333333,42.166667,36.000000,98.435897,3.000000,3.000000,4.333333,NaN,NaN,NaN,18.000000,NaN,NaN,108.000000,0.800000,153.750000,9.500000,1.45,121.500000,4.088889,137.500000,17.500000,32.750000,Coronary atherosclerosis of native coronary ar...,"lete, apnea, drift back to sleep, obe, hypovol...",5.2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14399,53.7,22.461314,68.940666,152.684211,55.041509,37.954350,97.714286,3.112676,1.706714,4.616197,67.0,47.5,3.44,34.793103,NaN,NaN,105.275862,0.927586,161.782609,10.844444,1.10,448.740741,4.330303,139.896552,16.648148,26.382609,Acute respiratory failure; Atrial fibrillation...,"hypertension, withdraw pain grim weakness spon...",22.6,0
14400,53.8,23.518519,61.888889,131.423077,66.192308,37.230000,96.923077,2.000000,1.000000,1.857143,475.0,269.5,3.70,82.000000,NaN,NaN,107.000000,1.350000,159.000000,10.800000,1.20,457.000000,4.650000,147.000000,14.150000,25.700000,"Acute kidney failure, unspecified; Candidiasis...","gall altered mental l gall adenomatosis, duct ...",5.8,0
14401,44.7,16.694737,81.526316,115.500000,71.904762,37.500000,97.543478,3.411765,4.058824,5.647059,NaN,NaN,NaN,17.000000,NaN,NaN,106.000000,1.150000,116.473684,10.700000,1.20,174.400000,4.041667,139.600000,14.220000,40.433333,"Accidental cut, puncture, perforation or hemor...","sl restless anxiety bs rash letgic, agitated a...",7.4,0
14402,81.0,16.178082,75.050000,128.531250,69.875000,35.776000,98.287500,3.461538,3.153846,5.615385,18.0,40.0,3.90,22.250000,NaN,NaN,102.500000,1.250000,107.133333,10.840000,1.15,158.000000,4.493750,134.000000,8.380000,52.316667,Abdominal aneurysm without mention of rupture;...,"pain ecy, ed, ested cough, utum, soft void uri...",6.0,0


In [23]:
dict_transfor = {
'age': 'Age (year)',
'Respiratory rate': 'Respiratory rate (min)',
'Heart rate': 'Heart rate (min)',
'Systolic blood pressure': 'SBP (mmHg)',
'Diastolic blood pressure': 'DBP (mmHg)',
'Body temperature': 'Body temperature (℃)',
'Saturation of peripheral oxygen': 'SpO2 (%)',
'Glasgow coma scale - eye': 'GCS–eye',
'Glasgow coma scale - verbal': 'GCS–verbal',
'Glasgow coma scale - motor': 'GCS–motor',
'Alanine aminotransferase': 'ALT (Units/L)',
'Aspartate aminotransferase': 'AST (Units/L)',
'Albumin': 'Albumin (g/dL)',
'Blood urea nitrogen': 'BUN (mg/dL)',
'Bilirubin': 'Bilirubin (mg/dL)',
'C-reactive protein': 'CRP (mg/dL)',
'Chloride': 'Chloride (mmol/L)',
'Creatinine': 'Creatinine (mg/dL)',
'Glucose': 'Glucose (mg/dL)',
'Hemoglobin': 'Hemoglobin (g/dL)',
'Prothrombin time (INR)': 'Prothrombin time (INR)',
'Platelets': 'Platelets (10^3/µL)',
'Potassium': 'Potassium (mmol/L)',
'Sodium': 'Sodium (mmol/L)',
'White blood cell': 'WBC (10^3/µL)',
'Activated partial thromboplastin time': 'aPTT (sec)',
'icd_title':'ICD text',
'Chief Complaint':'Chief complaint',
'hospital_days':'ICU_stay',
'hospital_expire_flag': 'Label'
}
df4 = df_merged4.rename(columns=dict_transfor)
df4['ICU_stay'] = np.ceil(df4['ICU_stay'])
df4['ICU_stay'] = df4['ICU_stay'].astype(int)
df4

,Age (year),Respiratory rate (min),Heart rate (min),SBP (mmHg),DBP (mmHg),Body temperature (℃),SpO2 (%),GCS–eye,GCS–verbal,GCS–motor,ALT (Units/L),AST (Units/L),Albumin (g/dL),BUN (mg/dL),Bilirubin (mg/dL),CRP (mg/dL),Chloride (mmol/L),Creatinine (mg/dL),Glucose (mg/dL),Hemoglobin (g/dL),Prothrombin time (INR),Platelets (10^3/µL),Potassium (mmol/L),Sodium (mmol/L),WBC (10^3/µL),aPTT (sec),long_title,Chief complaint,ICU_stay,Label
0,41.8,17.542857,83.771429,133.588235,71.000000,36.096000,98.285714,1.000000,1.000000,1.083333,NaN,NaN,NaN,30.500000,NaN,NaN,102.500000,1.850000,149.000000,12.050000,1.10,318.000000,3.600000,133.400000,11.650000,23.300000,"Acute kidney failure, unspecified; Congestive ...","awake, pain fl, ec, coughing vomited, vomit, e...",5,1
1,50.1,18.176471,84.918919,106.666667,54.194444,36.768750,96.655172,4.000000,4.941176,5.882353,NaN,NaN,NaN,16.500000,NaN,NaN,106.000000,0.700000,128.333333,12.550000,NaN,250.500000,3.850000,141.500000,11.050000,NaN,Malignant neoplasm of parietal lobe,"meta, meta meta lift hold discomfort b, masses...",26,0
2,39.9,17.646552,98.017241,107.250000,48.500000,37.122500,98.592920,3.782609,4.304348,5.608696,16.0,16.0,3.90,14.833333,NaN,NaN,107.200000,0.583333,158.833333,10.820000,1.40,135.500000,4.175000,138.375000,17.200000,33.875000,Coronary atherosclerosis of native coronary ar...,"anxiety pain pain, diminished breath collapse ...",7,0
3,91.0,20.583333,98.416667,146.708333,62.000000,36.872857,98.208333,4.000000,4.200000,6.000000,NaN,NaN,NaN,25.000000,NaN,NaN,105.000000,1.200000,139.000000,12.800000,1.30,216.000000,4.200000,138.000000,10.100000,30.400000,Accidental fall on or from other stairs or ste...,"ur, pain, ub cat, abrasions, at mass, trauma, ...",6,0
4,75.9,15.512821,79.121951,118.333333,42.166667,36.000000,98.435897,3.000000,3.000000,4.333333,NaN,NaN,NaN,18.000000,NaN,NaN,108.000000,0.800000,153.750000,9.500000,1.45,121.500000,4.088889,137.500000,17.500000,32.750000,Coronary atherosclerosis of native coronary ar...,"lete, apnea, drift back to sleep, obe, hypovol...",6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14399,53.7,22.461314,68.940666,152.684211,55.041509,37.954350,97.714286,3.112676,1.706714,4.616197,67.0,47.5,3.44,34.793103,NaN,NaN,105.275862,0.927586,161.782609,10.844444,1.10,448.740741,4.330303,139.896552,16.648148,26.382609,Acute respiratory failure; Atrial fibrillation...,"hypertension, withdraw pain grim weakness spon...",23,0
14400,53.8,23.518519,61.888889,131.423077,66.192308,37.230000,96.923077,2.000000,1.000000,1.857143,475.0,269.5,3.70,82.000000,NaN,NaN,107.000000,1.350000,159.000000,10.800000,1.20,457.000000,4.650000,147.000000,14.150000,25.700000,"Acute kidney failure, unspecified; Candidiasis...","gall altered mental l gall adenomatosis, duct ...",6,0
14401,44.7,16.694737,81.526316,115.500000,71.904762,37.500000,97.543478,3.411765,4.058824,5.647059,NaN,NaN,NaN,17.000000,NaN,NaN,106.000000,1.150000,116.473684,10.700000,1.20,174.400000,4.041667,139.600000,14.220000,40.433333,"Accidental cut, puncture, perforation or hemor...","sl restless anxiety bs rash letgic, agitated a...",8,0
14402,81.0,16.178082,75.050000,128.531250,69.875000,35.776000,98.287500,3.461538,3.153846,5.615385,18.0,40.0,3.90,22.250000,NaN,NaN,102.500000,1.250000,107.133333,10.840000,1.15,158.000000,4.493750,134.000000,8.380000,52.316667,Abdominal aneurysm without mention of rupture;...,"pain ecy, ed, ested cough, utum, soft void uri...",6,0


In [24]:
df4.to_csv("journalCC_MIMIC3_subset_StruWithNonStru_renamed_ICUstay.csv",index=False)